In [102]:
import pandas as pd
import numpy as np
import ast
import json
import urllib
import urllib2
import fiona
from fiona import crs
from cStringIO import StringIO

In [103]:
# Query Function from Data notebook
SQL_SOURCE = 'https://htv210.cartodb.com/api/v2/sql'

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())  

In [104]:
query = 'select zipcode from nyc_zipcode_highway'# LIMIT 5'
stream = queryCartoDB(query, format='CSV')
zipcodes       = pd.read_csv(stream)

In [109]:
zipcodes['zipcode']

AttributeError: 'Series' object has no attribute 'contains'

In [75]:
def convertZip(zip):
    try:
        return int(zip)
    except:
        return 0

In [114]:
subset = pd.read_csv('311.csv', converters = {'Incident Zip': convertZip})
#subset = subset[(subset['Incident Zip'] > 10000) & (subset['Incident Zip'] < 11693)]
subset = subset[subset['Incident Zip'].isin(zipcodes['zipcode'])]

In [115]:
subset['dt'] = pd.to_datetime(subset['Created Date'], format='%m/%d/%Y %I:%M:%S %p')

In [116]:
subset['year'] = subset.dt.dt.year

In [117]:
subset

,Created Date,Descriptor,Incident Zip,dt,year
0,12/31/2013 11:54:04 PM,Smoking Violation,11218,2013-12-31 23:54:04,2013
1,12/31/2013 09:00:00 PM,Lead Kit Request (Residential) (L10),10001,2013-12-31 21:00:00,2013
2,12/31/2013 08:44:00 PM,Noise: Construction Equipment (NC1),10033,2013-12-31 20:44:00,2013
3,12/31/2013 08:26:00 PM,Sewer Backup (Use Comments) (SA),11436,2013-12-31 20:26:00,2013
4,12/31/2013 07:35:00 PM,Sewer Backup (Use Comments) (SA),11212,2013-12-31 19:35:00,2013
5,12/31/2013 06:41:00 PM,Noise: Construction Before/After Hours (NM1),11215,2013-12-31 18:41:00,2013
6,12/31/2013 06:37:00 PM,Sewer Backup (Use Comments) (SA),11223,2013-12-31 18:37:00,2013
7,12/31/2013 06:35:00 PM,Noise: Construction Before/After Hours (NM1),11215,2013-12-31 18:35:00,2013
9,12/31/2013 06:17:00 PM,Noise: Construction Before/After Hours (NM1),11215,2013-12-31 18:17:00,2013
10,12/31/2013 06:14:00 PM,Noise: Construction Before/After Hours (NM1),11215,2013-12-31 18:14:00,2013


In [118]:
#all = pd.read_csv('311all.csv', dtype={'Incident Zip': np.float64}, error_bad_lines=False)
all = pd.read_csv('311all.csv', converters = {'Incident Zip': convertZip})
all = all[all['Incident Zip'].isin(zipcodes['zipcode'])]
#all = all[all['Incident Zip'] > 10000]

In [98]:
#all = all[all['Incident Zip'] < 11693]

In [119]:
all['dt'] = pd.to_datetime(all['Created Date'], format='%m/%d/%Y %I:%M:%S %p')
all['year'] = all.dt.dt.year

In [120]:
all

,Created Date,Descriptor,Incident Zip,dt,year
0,12/31/2013 11:59:00 PM,Catch Basin Clogged/Flooding (Use Comments) (SC),11414,2013-12-31 23:59:00,2013
1,12/31/2013 11:57:00 PM,Catch Basin Clogged/Flooding (Use Comments) (SC),11414,2013-12-31 23:57:00,2013
2,12/31/2013 11:56:54 PM,Loud Music/Party,11429,2013-12-31 23:56:54,2013
3,12/31/2013 11:54:26 PM,No Access,11369,2013-12-31 23:54:26,2013
4,12/31/2013 11:54:04 PM,Smoking Violation,11218,2013-12-31 23:54:04,2013
5,12/31/2013 11:52:00 PM,Leak (Use Comments) (WA2),10312,2013-12-31 23:52:00,2013
6,12/31/2013 11:49:51 PM,Blocked Sidewalk,11206,2013-12-31 23:49:51,2013
7,12/31/2013 11:49:00 PM,Veh Signal Lamp,11203,2013-12-31 23:49:00,2013
8,12/31/2013 11:48:41 PM,Posted Parking Sign Violation,11220,2013-12-31 23:48:41,2013
10,12/31/2013 11:46:43 PM,Loud Music/Party,10022,2013-12-31 23:46:43,2013


In [123]:
subset_group = subset.groupby(('year', 'Incident Zip'))
subset_count = subset_group.count()

all_group = all.groupby(('year', 'Incident Zip'))
all_count = all_group.count()

In [36]:
df = pd.DataFrame({
    
})

Incident Zip
6604       1
10000      2
10001    354
10002    650
10003    770
10004     96
10005    105
10006    121
10007    285
10009    592
10010    294
10011    713
10012    405
10013    558
10014    507
10016    709
10017    264
10018    197
10019    567
10020      4
10021    427
10022    522
10023    602
10024    627
10025    912
10026    286
10027    539
10028    479
10029    522
10030    245
        ... 
11379    224
11385    692
11411    259
11412    554
11413    449
11414    198
11415    170
11416    162
11417    192
11418    259
11419    391
11420    404
11421    220
11422    318
11423    424
11426    193
11427    192
11428    230
11429    370
11430      2
11432    625
11433    764
11434    726
11435    669
11436    381
11691    961
11692    288
11693    128
11694    225
11697      8
Name: dt, dtype: int64

In [89]:
subset_count['dt'][2010]/all_count['dt'][2010]

Incident Zip
10001        0.051959
10002        0.051073
10003        0.064880
10004        0.072562
10005        0.085784
10006        0.144391
10007        0.112072
10008             NaN
10009        0.058742
10010        0.063636
10011        0.069738
10012        0.058662
10013        0.061064
10014        0.058036
10016        0.082798
10017        0.064532
10018        0.057788
10019        0.054227
10020        0.015810
10021        0.074171
10022        0.072399
10023        0.076532
10024        0.069923
10025        0.071936
10026        0.041318
10027        0.047881
10028        0.076347
10029        0.052132
10030        0.038577
10031        0.030649
               ...   
90024             NaN
90049             NaN
91309             NaN
91716             NaN
92025             NaN
92123             NaN
92193             NaN
92618             NaN
92619             NaN
92701             NaN
92705             NaN
92780             NaN
94524             NaN
94551             N

In [47]:
all_count['dt'][2010].index

Index([u'*', u'.', u'00', u'000', u'0000', u'0000 000', u'0000 0000',
       u'0000-000', u'0000-0000', u'00000', 
       ...
       u'999999', u'?', u'MEXICO7500', u'NJ 07114', u'NY', u'PE60ND', u'SS',
       u'UNKNOWN', u'X', u'XXXXX'],
      dtype='object', name=u'Incident Zip', length=886)

In [53]:
all['Incident Zip'].isnumeric()

AttributeError: 'Series' object has no attribute 'isnumeric'

In [61]:
np.core.defchararray.isnumeric(all['Incident Zip'].astype('U'))

TypeError: isnumeric is only available for Unicode strings and arrays

In [57]:
np.isfinite(all['Incident Zip'])

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [65]:
all['Incident Zip'].astype('unicode')

0          11414
1          11414
2          11429
3          11369
4          11218
5          10312
6          11206
7          11203
8          11220
9            nan
10         10022
11         11414
12         11230
13         10466
14         11235
15         10468
16         11372
17         11103
18         10031
19         11103
20         11368
21         11365
22         11103
23           nan
24         10002
25         10033
26         10033
27         11356
28         10034
29         11372
           ...  
6624213    10040
6624214    11355
6624215    10463
6624216    11225
6624217    11229
6624218    11226
6624219    10025
6624220    11230
6624221    11225
6624222    10012
6624223    11233
6624224    10306
6624225    10457
6624226    11226
6624227    10468
6624228    10460
6624229    11355
6624230    10455
6624231    10472
6624232    10019
6624233    10472
6624234    11210
6624235    10035
6624236    11216
6624237    11220
6624238    10472
6624239    10455
6624240    103

In [121]:
len(np.unique(subset['Incident Zip']))

180

In [122]:
len(np.unique(all['Incident Zip']))

185

In [113]:
subset[subset['Incident Zip'].isin(zipcodes['zipcode'])]

,Created Date,Descriptor,Incident Zip
0,12/31/2013 11:54:04 PM,Smoking Violation,11218
1,12/31/2013 09:00:00 PM,Lead Kit Request (Residential) (L10),10001
2,12/31/2013 08:44:00 PM,Noise: Construction Equipment (NC1),10033
3,12/31/2013 08:26:00 PM,Sewer Backup (Use Comments) (SA),11436
4,12/31/2013 07:35:00 PM,Sewer Backup (Use Comments) (SA),11212
5,12/31/2013 06:41:00 PM,Noise: Construction Before/After Hours (NM1),11215
6,12/31/2013 06:37:00 PM,Sewer Backup (Use Comments) (SA),11223
7,12/31/2013 06:35:00 PM,Noise: Construction Before/After Hours (NM1),11215
9,12/31/2013 06:17:00 PM,Noise: Construction Before/After Hours (NM1),11215
10,12/31/2013 06:14:00 PM,Noise: Construction Before/After Hours (NM1),11215


In [112]:
subset

,Created Date,Descriptor,Incident Zip
0,12/31/2013 11:54:04 PM,Smoking Violation,11218
1,12/31/2013 09:00:00 PM,Lead Kit Request (Residential) (L10),10001
2,12/31/2013 08:44:00 PM,Noise: Construction Equipment (NC1),10033
3,12/31/2013 08:26:00 PM,Sewer Backup (Use Comments) (SA),11436
4,12/31/2013 07:35:00 PM,Sewer Backup (Use Comments) (SA),11212
5,12/31/2013 06:41:00 PM,Noise: Construction Before/After Hours (NM1),11215
6,12/31/2013 06:37:00 PM,Sewer Backup (Use Comments) (SA),11223
7,12/31/2013 06:35:00 PM,Noise: Construction Before/After Hours (NM1),11215
8,12/31/2013 06:33:00 PM,Sewer Backup (Use Comments) (SA),0
9,12/31/2013 06:17:00 PM,Noise: Construction Before/After Hours (NM1),11215


In [129]:
subset_count

Created Date  Descriptor   dt
year Incident Zip                               
2010 10001                  354         354  354
     10002                  650         650  650
     10003                  770         770  770
     10004                   96          96   96
     10005                  105         105  105
     10006                  121         121  121
     10007                  285         285  285
     10009                  592         592  592
     10010                  294         294  294
     10011                  713         713  713
     10012                  405         405  405
     10013                  558         558  558
     10014                  507         507  507
     10016                  709         709  709
     10017                  264         264  264
     10018                  197         197  197
     10019                  567         567  567
     10020                    4           4    4
     10021                  427         427  427
     10022                  522         522  522
     10023                  602         602  602
     10024                  627         627  627
     10025                  912         912  912
     10026                  286         286  286
     10027                  539         539  539
     10028                  479         479  479
     10029                  522         522  522
     10030                  245         245  245
     10031                  604         604  604
     10032                  636         636  636
...                         ...         ...  ...
2013 11379                  181         181  181
     11385                  709         709  709
     11411                  195         195  195
     11412                  462         462  462
     11413                  331         331  331
     11414                  230         230  230
     11415                  230         230  230
     11416                  190         190  190
     11417                  217         217  217
     11418                  267         267  267
     11419                  342         342  342
     11420                  385         385  385
     11421                  219         219  219
     11422                  203         203  203
     11423                  284         284  284
     11426                  181         181  181
     11427                  209         209  209
     11428                  228         228  228
     11429                  309         309  309
     11430                    4           4    4
     11432                  518         518  518
     11433                  690         690  690
     11434                  730         730  730
     11435                  569         569  569
     11436                  291         291  291
     11691                  888         888  888
     11692                  384         384  384
     11693                  266         266  266
     11694                  355         355  355
     11697                   18          18   18

[716 rows x 3 columns]

In [139]:
subset_df = pd.DataFrame({
    'zip': subset_count['dt'][2010].index,
    '2010_s': subset_count['dt'][2010]    
})

for i in range(2011,2014):
    d2 = pd.DataFrame({
        'zip': subset_count['dt'][i].index,
        '%s_s' % i: subset_count['dt'][i]    
    })
    subset_df = subset_df.merge(d2, on='zip', how='outer')

In [141]:
all_df = pd.DataFrame({
    'zip': all_count['dt'][2010].index,
    '2010_a': all_count['dt'][2010]    
})

for i in range(2011,2014):
    d2 = pd.DataFrame({
        'zip': all_count['dt'][i].index,
        '%s_a' % i: all_count['dt'][i]    
    })
    all_df = all_df.merge(d2, on='zip', how='outer')

In [143]:
df = all_df.merge(subset_df, on='zip', how='outer')

In [144]:
df

,2010_a,zip,2011_a,2012_a,2013_a,2010_s,2011_s,2012_s,2013_s
0,6813,10001,6201,5867,6841,354,391,395,749
1,12727,10002,11372,10834,11328,650,586,633,661
2,11868,10003,11191,10703,11286,770,733,672,820
3,1323,10004,1344,1488,1541,96,91,97,173
4,1224,10005,1400,1693,1832,105,170,222,355
5,838,10006,1024,740,781,121,174,115,198
6,2543,10007,2373,2182,2451,285,304,276,324
7,10078,10009,8971,9437,9645,592,508,597,699
8,4620,10010,4412,4463,4850,294,292,372,509
9,10224,10011,9508,8809,10012,713,651,760,967


In [145]:
df.fillna(0, inplace=True)

In [148]:
df['2010_r'] = df['2010_s']/df['2010_a']
df['2011_r'] = df['2011_s']/df['2011_a']
df['2012_r'] = df['2012_s']/df['2012_a']
df['2013_r'] = df['2013_s']/df['2013_a']

In [154]:
df['total_a'] = df['2010_a'] + df['2011_a'] + df['2012_a'] + df['2013_a']
df['total_s'] = df['2010_s'] + df['2011_s'] + df['2012_s'] + df['2013_s']
df['total_r'] = df['total_s']/df['total_a']

In [155]:
df

,2010_a,zip,2011_a,2012_a,2013_a,2010_s,2011_s,2012_s,2013_s,2010_r,2011_r,2012_r,2013_r,total_a,total_s,total_r
0,6813,10001,6201,5867,6841,354,391,395,749,0.051959,0.063054,0.067326,0.109487,25722,1889,0.073439
1,12727,10002,11372,10834,11328,650,586,633,661,0.051073,0.051530,0.058427,0.058351,46261,2530,0.054690
2,11868,10003,11191,10703,11286,770,733,672,820,0.064880,0.065499,0.062786,0.072656,45048,2995,0.066485
3,1323,10004,1344,1488,1541,96,91,97,173,0.072562,0.067708,0.065188,0.112265,5696,457,0.080232
4,1224,10005,1400,1693,1832,105,170,222,355,0.085784,0.121429,0.131128,0.193777,6149,852,0.138559
5,838,10006,1024,740,781,121,174,115,198,0.144391,0.169922,0.155405,0.253521,3383,608,0.179722
6,2543,10007,2373,2182,2451,285,304,276,324,0.112072,0.128108,0.126489,0.132191,9549,1189,0.124516
7,10078,10009,8971,9437,9645,592,508,597,699,0.058742,0.056627,0.063262,0.072473,38131,2396,0.062836
8,4620,10010,4412,4463,4850,294,292,372,509,0.063636,0.066183,0.083352,0.104948,18345,1467,0.079967
9,10224,10011,9508,8809,10012,713,651,760,967,0.069738,0.068469,0.086275,0.096584,38553,3091,0.080175


In [156]:
df.to_csv('311_processed.csv', index=False)